# Welcome to OlympusDAO Playground

## Introduction: What is Olympus Playground?

This is a simple projection calculator for Staking and bonding ohm in OlympusDAO protocol
Welcome to Olympus Playground
This is an interactive notebook to study, play and forcast the growth of your ohm over time. This notebook is designed to work hand in hand with the awesome gitbook created by the DAO and all of Brians calcs!

### Staking Projections:

- ROIs at current reward yield
- ROI after 1 day at current reward yield
- ROI after 5 days at current reward yield
- OHM and USD ROI and holding value in 5 days
- ROI after 7 days at current reward yield
- OMH and USD ROI and holding value in 7 days
- ROI after 1 month at current reward yield
- OHM and USD ROI and holding value in 1 Month
- ROI after 1 year at current reward yield
- OHM and USD ROI and holding value in 1 Year
- Days until desired target USD
- Days until desired target OHM
- OHM and USD ROI and holding on specific date. This data will be provided on an interactive chart where you can hover your mouse over any desired date and view the projection.

### Staking Rewards Projections:

- Required OHM to reach desired daily incooom
- Days until you reach required OHMs for daily incoom (Assuming growth by staking alone)
- Estimated date
Desire weekly incooom
- Required OHM to reach goal desire weekly incooom
- weeks until you reach required OHMs for weeks incoom (Assuming growth by staking alone)
- Estimated date


# The Libraries

In [151]:
# ==============THE LIBRARIES
# region Description: Import all required libraries for this simulator
from pycoingecko import CoinGeckoAPI # Coin gecko API: Pulls live data from coin gecko
import math  # Needed for basic math operations\n",
import pandas as pd  # Needed fpr dataframe creation and operations\n",
import numpy as np  # Needed for array manipulations\n",
from itertools import islice  # Needed for more complex row and coloumn slicing\n",
import matplotlib.pyplot as plt  # Needed for quickly ploting results"
import base64
from PIL import Image
import pathlib  # url management
from pathlib import Path
import plotly.express as px  # cleaner graphs
import plotly.graph_objects as go  # cleaner graphs
import plotly.figure_factory as ff
import streamlit as st
import requests

### Streamlit Library

Streamlit is an open-source Python library that makes it easy to create and share beautiful, custom web apps for machine learning and data science

https://docs.streamlit.io/

### Plotly Library

Plotly's Python graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.

https://plotly.com/python/

### Pandas Library

pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language

https://pandas.pydata.org/

### Numpy Library

NumPy is the fundamental package for scientific computing in Python. 
It is a Python library that provides a multidimensional array object, various derived objects 
(such as masked arrays and matrices), and an assortment of routines for fast operations on arrays, including mathematical, logical, shape manipulation, sorting, selecting, I/O, discrete Fourier transforms, basic linear algebra, basic statistical operations, random simulation and much more.

https://numpy.org/

# The server 

Playgrounds is hosted on the streamlit cloud server

https://streamlit.io/cloud#plans-table

All py files are stored and read from the Playgrounds github repo
https://github.com/Tachikoma000/Olympus_Playground

# Core Code

## Staking Simulator

The staking simulator is comprised of two sections:

        -Protocol Metrics scanner 
        -Ohm growth forecast
        -Income (staking rewards) forecast

### Protocol Metrics Scanner


In [5]:
def run_query(q):

        # endpoint where you are making the request
        request = requests.post('https://api.thegraph.com/subgraphs/name/drondin/olympus-graph'
                                '',
                                json={'query': query})
        if request.status_code == 200:
            return request.json()
        else:
            raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

    # The Graph query... these queries are made available to us on the olympus sub graph -
    query = """
        {
        protocolMetrics(first: 1, orderBy: timestamp, orderDirection: desc) {
            timestamp
            ohmCirculatingSupply
            sOhmCirculatingSupply
            totalSupply
            ohmPrice
            marketCap
            totalValueLocked
            treasuryMarketValue
            treasuryRiskFreeValue
            runwayCurrent
            currentAPY
            nextEpochRebase
            nextDistributedOhm
          }
        }
        """
    result = run_query(query)

    # results come as a list of dictionaries, which is in a nested dictionary. so we have to extract the dictionary we care about
    dataDict = result['data']['protocolMetrics']
    protocolMetrics_df = pd.DataFrame(dataDict)
    protocolMetrics_df = protocolMetrics_df.astype(float)
    protocolMetrics_df['dateTime'] = pd.to_datetime(protocolMetrics_df.timestamp, unit='s')
    protocolMetrics_df['dateTime'] = protocolMetrics_df['dateTime'].dt.date
    protocolMetrics_df = protocolMetrics_df.set_index('dateTime')
    #st.write(protocolMetrics_df)

    currentOhmPrice = round(float(protocolMetrics_df.ohmPrice.iloc[0]),2)
    currentOCirc = round(float(protocolMetrics_df.ohmCirculatingSupply.iloc[0]), 2)
    currentSOCirc = round(float(protocolMetrics_df.sOhmCirculatingSupply.iloc[0]), 2)
    currentTSupply = round(float(protocolMetrics_df.totalSupply.iloc[0]), 2)
    currentMCap = round(float(protocolMetrics_df.marketCap.iloc[0]), 2)
    currentTVL = round(float(protocolMetrics_df.totalValueLocked.iloc[0]),2)
    currentTMV = round(float(protocolMetrics_df.treasuryMarketValue.iloc[0]), 2)
    currentTRFV = round(float(protocolMetrics_df.treasuryRiskFreeValue.iloc[0]), 2)
    currentRunway = round(float(protocolMetrics_df.runwayCurrent.iloc[0]), 2)
    currentAPY = round(float(protocolMetrics_df.currentAPY.iloc[0]), 2)
    currentEpochs = round(float(protocolMetrics_df.nextEpochRebase.iloc[0]), 4)
    currentNextDist = round(float(protocolMetrics_df.nextDistributedOhm.iloc[0]), 2)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 13)

###  Section 1: Ohm growth forecast

In [2]:
def ohmGrowth_Projection(initialOhms, userAPY, ohmGrowthDays, minAPY, maxAPY,percentSale, sellDays, ohmPrice_DCA, valBuy, buyDays):

    # Data frame to hold all required data point. Data required would be Epochs since rebase are distributed every Epoch
    ohmGrowthEpochs = (ohmGrowthDays * 3)+1
    sellEpochs = sellDays * 3
    buyEpochs = buyDays * 3
    cadenceConst = sellEpochs
    cadenceConst_BUY = buyEpochs
    dcaAmount = valBuy/ohmPrice_DCA
    percentSale = percentSale/100
    userAPY = userAPY/100
    minAPY = minAPY/100
    maxAPY = maxAPY/100


    rewardYield = ((1+userAPY)**(1/float(1095)))-1
    minOIPYield = ((1 + minAPY) ** (1 / float(1095))) - 1
    maxOIPYield = ((1 + maxAPY) ** (1 / float(1095))) - 1


    ohmGrowth_df = pd.DataFrame(np.arange(ohmGrowthEpochs),columns=['Epochs'])  # In this case let's consider 1096 Epochs which is 365 days
    ohmGrowth_df['Days'] = ohmGrowth_df.Epochs / 3  # There are 3 Epochs per day so divide by 3 to get Days

    profitAdjusted_ohmGrowth_df = pd.DataFrame(np.arange(ohmGrowthEpochs),columns=['Epochs'])
    profitAdjusted_ohmGrowth_df['Days'] = profitAdjusted_ohmGrowth_df.Epochs / 3

    dollarCostAVG_ohmGrowth_df = pd.DataFrame(np.arange(ohmGrowthEpochs),columns=['Epochs'])
    dollarCostAVG_ohmGrowth_df['Days'] = profitAdjusted_ohmGrowth_df.Epochs / 3

    # To Calculate the ohm growth over 3000 Epochs or 1000 days, we loop through the exponential ohm growth equation every epoch
    totalOhms = []  # create an empty array that will hold the componded rewards
    pA_totalOhms = []
    dcA_totalOhms = []

    rewardYield = round(rewardYield, 5)
    ohmStakedGrowth = initialOhms  # Initial staked ohms used to project growth over time
    pA_ohmStakedGrowth = initialOhms
    dcA_ohmStakedGrowth = initialOhms


    # Initialize the for loop to have loops equal to number of rows or number of epochs
    for elements in ohmGrowth_df.Epochs:
        totalOhms.append(ohmStakedGrowth)  # populate the empty array with calclated values each iteration
        pA_totalOhms.append(pA_ohmStakedGrowth)
        dcA_totalOhms.append(dcA_ohmStakedGrowth)


        ohmStakedGrowth = ohmStakedGrowth * (1 + rewardYield)  # compound the total amount of ohms
        pA_ohmStakedGrowth = pA_ohmStakedGrowth * (1 + rewardYield)
        dcA_ohmStakedGrowth = dcA_ohmStakedGrowth * (1 + rewardYield)

        if elements == sellEpochs:
            sellEpochs = sellEpochs + cadenceConst
            #print(totalOhms[-1] - (totalOhms[-1] * percentSale))
            pA_ohmStakedGrowth = pA_totalOhms[-1] - (pA_totalOhms[-1]*percentSale)
        else:
            pass

        if elements == buyEpochs:
            buyEpochs = buyEpochs + cadenceConst_BUY
            #print(dcA_ohmStakedGrowth)
            dcA_ohmStakedGrowth = dcA_ohmStakedGrowth + dcaAmount
            #print(dcA_ohmStakedGrowth)
        else:
            pass

    ohmGrowth_df['Total_Ohms'] = totalOhms  # Clean up and add the new array to the main data frame
    ohmGrowth_df['Profit_Adjusted_Total_Ohms'] = pA_totalOhms
    ohmGrowth_df['DCA_Adjusted_Total_Ohms'] = dcA_totalOhms
    ohmGrowth_df.Days = np.around( ohmGrowth_df.Days, decimals=1)  # Python is funny so let's round up our numbers . 1 decimal place for days",
    ohmGrowth_df.Total_Ohms = np.around( ohmGrowth_df.Total_Ohms, decimals=3 )  # Python is funny so let's round up our numbers . 3 decimal place for ohms"
    ohmGrowth_df.Profit_Adjusted_Total_Ohms = np.around(ohmGrowth_df.Profit_Adjusted_Total_Ohms, decimals=3)
    ohmGrowth_df.DCA_Adjusted_Total_Ohms = np.around(ohmGrowth_df.DCA_Adjusted_Total_Ohms, decimals=3)


    totalOhms_minOIPRate = []
    minOIPYield = round(minOIPYield, 5)
    ohmStakedGrowth_minOIPRate = initialOhms  # Initial staked ohms used to project growth over time
    # Initialize the for loop to have loops equal to number of rows or number of epochs
    for elements in ohmGrowth_df.Epochs:
        totalOhms_minOIPRate.append(
            ohmStakedGrowth_minOIPRate)  # populate the empty array with calclated values each iteration
        ohmStakedGrowth_minOIPRate = ohmStakedGrowth_minOIPRate * (1 + minOIPYield)  # compound the total amount of ohms
    ohmGrowth_df['Min_OhmGrowth'] = totalOhms_minOIPRate  # Clean up and add the new array to the main data frame

    totalOhms_maxOIPRate = []
    maxOIPYield = round(maxOIPYield, 5)
    ohmStakedGrowth_maxOIPRate = initialOhms  # Initial staked ohms used to project growth over time
    # Initialize the for loop to have loops equal to number of rows or number of epochs
    for elements in ohmGrowth_df.Epochs:
        totalOhms_maxOIPRate.append(
            ohmStakedGrowth_maxOIPRate)  # populate the empty array with calclated values each iteration
        ohmStakedGrowth_maxOIPRate = ohmStakedGrowth_maxOIPRate * (1 + maxOIPYield)  # compound the total amount of ohms
    ohmGrowth_df['Max_OhmGrowth'] = totalOhms_maxOIPRate  # Clean up and add the new array to the main data frame

    ohmGrowth_df_CSV = ohmGrowth_df.to_csv().encode('utf-8')
    # ================================================================================


    return ohmGrowth_df,ohmGrowth_df_CSV
# end region


### Section2: Income (Staking rewards) forecast

In [3]:
def incooomProjection(ohmPrice,userAPY, initialOhms, desiredUSDTarget,desiredOHMTarget, desiredDailyIncooom,desiredWeeklyIncooom):

    ohmStakedInit = initialOhms
    userAPY = userAPY / 100
    rewardYield = ((1 + userAPY) ** (1 / float(1095))) - 1
    rewardYield = round(rewardYield, 5)
    rebaseConst = 1 + rewardYield
    # current staking %APY. Need to make this read from a source or user entry
    #currentAPY = 17407 / 100

    # Let's get some ROI Outputs starting with the daily
    dailyROI = (1+rewardYield)**3 -1  # Equation to calculate your daily ROI based on reward Yield
    dailyROI_P = round(dailyROI * 100, 1)  # daily ROI in Percentage
    # ================================================================================

    # 5 day ROI
    fivedayROI = (1+rewardYield)**(5*3)-1   # Equation to calculate your 5 day ROI based on reward Yield
    fivedayROI_P = round(fivedayROI * 100, 1)  # 5 day ROI in Percentage
    # ================================================================================

    # 7 day ROI
    sevendayROI = (1+rewardYield)**( 7 * 3)-1  # Equation to calculate your 7 day ROI based on reward Yield
    sevendayROI_P = round(sevendayROI * 100, 1)  # 7 day ROI in Percentage
    # ================================================================================

    # 30 day ROI
    monthlyROI = (1+rewardYield)**( 30 *3)-1  # Equation to calculate your 30 day ROI based on reward Yield
    monthlyROI_P = round(monthlyROI * 100, 1)  # 30 day ROI in Percentage
    # ================================================================================

    # Annual ROI
    annualROI = (1+rewardYield)**( 365 *3)-1  # Equation to calculate your annual ROI based on reward Yield
    annualROI_P = round(annualROI * 100, 1)  # Equation to calculate your annual ROI based on reward Yield
    # ================================================================================

    # Let's create a nice looking table to view the results of our calculations. The table will contain the ROIs and the percentages
    roiData = [['Daily', dailyROI_P],
               ['5 Day', fivedayROI_P],
               ['7 Day', sevendayROI_P],
               ['1 Month', monthlyROI_P],
               ['1 Year', annualROI_P]]
    roiTabulated_df = pd.DataFrame(roiData, columns=['Cadence', 'Percentage'])
    roiDataTable = roiTabulated_df.to_dict('rows')
    columns = [{'name': i,'id': i,} for i in (roiTabulated_df.columns)]
    # ================================================================================
    # Days until you reach target USD by staking only
    forcastUSDTarget = round((math.log(desiredUSDTarget / (ohmStakedInit * ohmPrice), rebaseConst) /3))
    # ================================================================================
    # Days until you reach target OHM by staking only
    forcastOHMTarget = round(math.log(desiredOHMTarget / (ohmStakedInit), rebaseConst) / 3)
    # ================================================================================
    # Daily Incooom calculations
    # Required OHMs until you are earning your desired daily incooom
    requiredOHMDailyIncooom = round((desiredDailyIncooom / dailyROI) / ohmPrice)
    # Days until you are earning your desired daily incooom from your current initial staked OHM amount
    forcastDailyIncooom = round(math.log((requiredOHMDailyIncooom / ohmStakedInit), rebaseConst) / 3)
    requiredUSDForDailyIncooom = requiredOHMDailyIncooom * ohmPrice
    # ================================================================================
    # Weekly Incooom calculations
    # Required OHMs until you are earning your desired weekly incooom
    requiredOHMWeeklyIncooom = round((desiredWeeklyIncooom / sevendayROI) / ohmPrice)
    # Days until you are earning your desired weekly incooom from your current initial staked OHM amount
    forcastWeeklyIncooom = round(math.log((requiredOHMWeeklyIncooom / ohmStakedInit), rebaseConst) / 3)
    requiredUSDForWeeklyIncooom = requiredOHMWeeklyIncooom * ohmPrice
    # ================================================================================
    # Let's create a nice looking table to view the results of our calculations. The table will contain the ROIs and the percentages
    incooomForcastData = [['USD Target($)', forcastUSDTarget],
                          ['OHM Target(OHM)', forcastOHMTarget],
                          ['Required OHM for desired daily incooom', requiredOHMDailyIncooom],
                          ['Days until desired daily incooom goal', forcastDailyIncooom],
                          ['Required OHM for weekly incooom goal', requiredOHMWeeklyIncooom],
                          ['Days until desired weekly incooom goal', forcastWeeklyIncooom]]

    incooomForcastData_df = pd.DataFrame(incooomForcastData, columns=['Forcast', 'Results'])
    incooomForcastDataDataTable = incooomForcastData_df.to_dict('rows')
    columns = [{'name': i,'id': i,} for i in (incooomForcastData_df.columns)]

    return roiTabulated_df,incooomForcastData_df, rewardYield

In [173]:
# Data frame to hold all required data point. Data required would be Epochs since rebase are distributed every Epoch
ohmGrowth_df = pd.DataFrame(np.arange(1096), columns=[
    'Epochs'
])  # In this case let's consider 1096 Epochs which is 365 days
ohmGrowth_df[
    'Days'] = ohmGrowth_df.Epochs / 3  # There are 3 Epochs per day so divide by 3 to get Days

In [174]:
# Data frame to hold OIP-18
# Start by creating a dictionary
oip18_dict = {
    'Total OHM supply range min': [
        '0', '1,000,000', '10,000,000', '100,000,000', '1,000,000,000',
        '10,000,000,000', '100,000,000,000'
    ],
    'Total OHM supply range max': [
        '1,000,000', '10,000,000', '100,000,000', '1,000,000,000',
        '10,000,000,000', '100,000,000,000', '1,000,000,000,000'
    ],
    'Min Reward Rate (%)':
    [0.3058, 0.1587, 0.1186, 0.0458, 0.0148, 0.0039, 0.0019],
    'Max Reward Rate (%)':
    [0.4583, 0.3058, 0.1587, 0.1186, 0.0458, 0.0148, 0.0039],
    'Min APY% (Assuming 90% Staked)': [10000, 1000, 500, 100, 25, 6, 3],
    'Max APY% (Assumung 90% Staked)': [100000, 10000, 1000, 500, 100, 25, 6],
}
# Then convert to pandas data frame
oip18_dataFrame = pd.DataFrame(oip18_dict)

In [175]:
# Data frame to hold all required data point for bonding simulations
bondingTable_Data = {'Scenarios': ['Base Case',
                                   'Staking before each Epoch',
                                   'Staking once a day',
                                   'Staking halfway through vesting (day 2.5)'],}

bondindSimulation_df = pd.DataFrame(bondingTable_Data)

## Bonding Simulator

The bonding simulator is comprised of one section

In [6]:
def bondingSimulation(ohmPrice,priceofETH,initialOhms,bondROI,rewardYield,gwei):
    # Protocol and ohm calcs:
    usdBonded = ohmPrice * initialOhms
    bondROI = (bondROI/100)
    bondPrice = ohmPrice / (1 + (bondROI))
    bondedOhms = usdBonded / bondPrice
    bondedOhmsValue = bondedOhms * ohmPrice
    # ========================================================================================
    # Calculate the rebase rate and Current APY (next epoch rebase pulled from hippo data source)
    rewardYield = rewardYield / 100
    rebaseConst = 1 + rewardYield  # calculate a constant for use in APY calculation
    currentAPY = (rebaseConst) ** (1095)   # current APY equation
    currentAPY_P = (currentAPY) * 100  # convert to %
    # ========================================================================================
    # Calculate fees
    stakingGasFee = 179123 * ((gwei * priceofETH) / (10 ** 9))
    unstakingGasFee = 89654 * ((gwei * priceofETH) / (10 ** 9))
    swappingGasFee = 225748 * ((gwei * priceofETH) / (10 ** 9)) + ((0.3 / 100) * bondedOhmsValue)
    claimGasFee = 80209 * ((gwei * priceofETH) / (10 ** 9))
    bondingGasFee = 258057 * ((gwei * priceofETH) / (10 ** 9))
    # miscFee = 823373 * ((gwei*priceofETH)/(10**9))
    # ================================================================================

    claimStakeGasFee = stakingGasFee + claimGasFee
    remainingGasFee = bondingGasFee + unstakingGasFee + swappingGasFee
    # ================================================================================
    # (3,3) Rate for the 15 epochs
    stakingRewardRate = (1+rewardYield) ** 15 - 1
    stakingRewardRate_P = round(stakingRewardRate*100,2)
    #stakingOhmsGained = round(((initOhmValue - stakingGasFee) * (stakingRate / initOhmValue) - 1),4)

    # (3,3) Ohm gained after 15 epochs
    stakingOhmGrowth = stakingRewardRate * bondedOhmsValue / ohmPrice
    stakingOhmGrowth = round(stakingOhmGrowth, 4)
    # ================================================================================
    vestedOhms_df = pd.DataFrame(np.arange(1, 16), columns=['Epochs'])
    vestedOhms_df['Days'] = vestedOhms_df.Epochs / 3
    vestedOhmGrowth = np.array([], dtype=np.float64)
    bondROIGrowth = np.array([], dtype=np.float64)

    stakedOhmsROI_df = pd.DataFrame(np.arange(1, 16), columns=['Epochs'])
    stakedOhmsROI_df['Days'] = stakedOhmsROI_df.Epochs / 3
    stakedROIAdjustedGrowth = np.array([], dtype=np.float64)
    stakeROIGrowth = np.array([], dtype=np.float64)
    stakedOhmsGrowth = np.array([], dtype=np.float64)
    stakeGrowth = initialOhms

    for epochs in vestedOhms_df.Epochs:
        vestedOhm = ((bondedOhms / (1 + epochs)) * (((1+rewardYield) ** 15) - 1)) / ((1+rewardYield) ** (15 / (1 + epochs)) - 1)
        vestedOhmROI = (((vestedOhm * ohmPrice - epochs * (claimStakeGasFee) - remainingGasFee) / usdBonded) - 1) * 100
        vestedOhmGrowth = np.append(vestedOhmGrowth, vestedOhm)
        bondROIGrowth = np.append(bondROIGrowth, vestedOhmROI)
    vestedOhms_df['Vested_Ohms'] = vestedOhmGrowth
    vestedOhms_df['Bond_ROI'] = bondROIGrowth

    for epochs in stakedOhmsROI_df.Epochs:
        stakedOhmsGrowth = np.append(stakedOhmsGrowth, stakeGrowth)
        stakedROIAdjusted = ((usdBonded - stakingGasFee) * (((1+rewardYield) ** 15) / usdBonded) - 1) * 100
        stakeROI = stakingRewardRate * 100
        stakeGrowth = stakeGrowth * (1+rewardYield)
        stakedROIAdjustedGrowth = np.append(stakedROIAdjustedGrowth, stakedROIAdjusted)
        stakeROIGrowth = np.append(stakeROIGrowth, stakeROI)
    stakedOhmsROI_df['Stake_ROI'] = stakeROIGrowth
    stakedOhmsROI_df['Staked_feeAdjustedROI'] = stakedROIAdjustedGrowth
    stakedOhmsROI_df['Stake_Growth'] = stakedOhmsGrowth
    # ================================================================================

    cols_to_use = stakedOhmsROI_df.columns.difference(vestedOhms_df.columns)
    stake_bond_df = pd.merge(vestedOhms_df, stakedOhmsROI_df[cols_to_use], left_index=True, right_index=True, how='outer')
    #stake_bond_df = pd.concat([vestedOhms_df,stakedOhmsROI_df],axis = 1, join = 'inner')

    maxBondROI = round(stake_bond_df.Bond_ROI.max(),2)
    maxStakeGrowth = round(stake_bond_df.Stake_Growth.max(),2)
    maxBondGrowth = round(stake_bond_df.Vested_Ohms.max(), 2)
    ohmGained = round((stake_bond_df.Vested_Ohms.max()-stake_bond_df.Stake_Growth.max()),2)
    stakingGasFee = round(stakingGasFee,2)
    unstakingGasFee = round(unstakingGasFee,2)
    swappingGasFee = round(swappingGasFee,2)
    claimGasFee = round(claimGasFee,2)
    bondingGasFee = round(bondingGasFee,2)

    vestedOhms_df_CSV = vestedOhms_df.to_csv().encode('utf-8')
    stakedOhmsROI_df_CSV = stakedOhmsROI_df.to_csv().encode('utf-8')

    return vestedOhms_df, stakedOhmsROI_df, stake_bond_df ,stakingGasFee,unstakingGasFee,swappingGasFee,claimGasFee,bondingGasFee,maxBondROI,stakingRewardRate_P,maxStakeGrowth,maxBondGrowth,ohmGained,vestedOhms_df_CSV,stakedOhmsROI_df_CSV

# Execute app

In [7]:
# App is executed on the streamlit server